# Proyecto de Día 11 - Análisis Predictivo para una Tienda Minorista

### Objetivo
Tu tarea consiste en desarrollar un **modelo predictivo** para una **tienda minorista** que desea pronosticar las ventas basándose en diversas características. La tienda ha estado recopilando datos sobre las ventas diarias y quiere utilizar esta información para tomar decisiones informadas sobre la *gestión de inventario*, *promociones* y *programación de personal*.

Tu objetivo es construir un modelo que pueda **predecir el total de ventas para el próximo mes**, utilizando datos de los meses anteriores.

### Tarea
**1. Preparación de Datos**
- Carga el conjunto de datos y realiza cualquier preprocesamiento necesario, como manejar **valores faltantes**, selección de **variables dependientes e independientes**, y dividir los datos en **conjuntos de entrenamiento** y **prueba**.


**2. Análisis Exploratorio de Datos (EDA)**
- Realiza un EDA para entender la distribución de las ventas y la relación entre las ventas y otras características. Esto podría incluir trazar las ventas a lo largo del tiempo, analizar las ventas en *festivos* vs. *no festivos* y el impacto de las promociones.

**3. Selección de Modelo**
- Utiliza el conocimiento que has adquirido sobre diferentes algoritmos para elegir el modelo más adecuado para esta tarea. Considera la **Regresión Lineal**, los **Árboles de Decisión**, y los **Bosques Aleatorios**.

**4. Entrenamiento y Evaluación del Modelo**
- Entrena tu modelo seleccionado en el conjunto de entrenamiento y evalúa su rendimiento utilizando el conjunto de prueba. Utiliza gráficos para visualizar las *ventas reales* vs. *las predichas*.

**5. Conclusión**

- Basado en los resultados iniciales, resume tus hallazgos y realiza recomendaciones para la tienda minorista sobre cómo utilizar el análisis predictivo para una mejor toma de decisiones.


### Nota
* Enfócate en la claridad y simplicidad de tu código.

¡Mucha Suerte!

### 0. Cargar librerías

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.linear_model as lm
import sklearn.tree as tr
import sklearn.model_selection as ms
import sklearn.preprocessing as sp
import sklearn.ensemble as se

### 1. Preparación de Datos

In [2]:
# Carga el conjunto de datos
df: pd.DataFrame = pd.read_csv('./data/Ventas.csv')
df

,Fecha,DíaDeLaSemana,Promociones,Festivo,Ventas
0,2022-01-01,6,0,0,4033
1,2022-01-02,7,0,1,4012
2,2022-01-03,1,0,0,1608
3,2022-01-04,2,1,0,2046
4,2022-01-05,3,0,0,2484
...,...,...,...,...,...
360,2022-12-27,2,1,0,2189
361,2022-12-28,3,0,0,2548
362,2022-12-29,4,0,0,3018
363,2022-12-30,5,0,0,3591


In [3]:
# Obtener información preliminar del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Fecha          365 non-null    object
 1   DíaDeLaSemana  365 non-null    int64 
 2   Promociones    365 non-null    int64 
 3   Festivo        365 non-null    int64 
 4   Ventas         365 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 14.4+ KB


In [4]:
# Obtener estadísticos generales del DataFrame
df.describe()

,DíaDeLaSemana,Promociones,Festivo,Ventas
count,365.000000,365.000000,365.000000,365.000000
mean,4.005479,0.200000,0.142466,2997.219178
std,2.002738,0.400549,0.350007,942.096284
min,1.000000,0.000000,0.000000,1305.000000
25%,2.000000,0.000000,0.000000,2195.000000
50%,4.000000,0.000000,0.000000,3074.000000
75%,6.000000,0.000000,0.000000,3979.000000
max,7.000000,1.000000,1.000000,4404.000000


El dataset entrega las ventas del negocio durante el año 2022. La columna `DíaDeLaSemana` indica el número del día de la semana en la que cae la fecha (`1` es lunes, `2` es martes, y así). Las columnas `Promociones` y `Festivo` parecen ser booleanos y simplemente indican si ese día hubo una promoción o fue un día festivo, respectivamente.

A primera vista, el dataset **no posee nulos**, por lo que se asume que todos los valores aquí proveídos son correctos (NOTA: Ok, el 1 de enero debería ser festivo y el 2 no, pero para efectos de este proyecto se puede dejar pasar, ya que no es el foco).

No obstante...

In [5]:
# Ver un elemento de la columna 'Fecha' y su tipo de dato
print(df['Fecha'][0])
print(type(df['Fecha'][0]))

2022-01-01
<class 'str'>


La columna `Fecha` es de tipo `str` (algo que siempre pasa al cargar el dataset desde CSV).

In [6]:
# Cambiar el tipo de dato de la columna 'Fecha' al tipo correcto
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d', dayfirst=True)
print(df['Fecha'][0])
print(type(df['Fecha'][0]))

2022-01-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


Dado que el objetivo es **predecir el total de ventas para el próximo mes**, se deduce que la columna `Ventas` será la _variable dependiente_, mientras que las demás cumplirán como _variables independientes_.

In [8]:
X: pd.DataFrame = df.drop('Ventas', axis=1)
X.head()

,Fecha,DíaDeLaSemana,Promociones,Festivo
0,2022-01-01,6,0,0
1,2022-01-02,7,0,1
2,2022-01-03,1,0,0
3,2022-01-04,2,1,0
4,2022-01-05,3,0,0


In [9]:
y: pd.DataFrame = df[['Ventas']]
y.head()

,Ventas
0,4033
1,4012
2,1608
3,2046
4,2484


In [10]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = ms.train_test_split(
    X, y, test_size=0.2, random_state=42
)